<a href="https://colab.research.google.com/github/drpetros11111/cv2_ssd/blob/main/funny_dog_detection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the libraries
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

# Importing the libraries
import torch from torch.autograd import Variable import cv2 from data import BaseTransform, VOC_CLASSES as labelmap from ssd import build_ssd import imageio

---
---


Okay, let's break down this code snippet line by line:

##import torch:
This line imports the PyTorch library, which is a popular deep learning framework.

It provides tools for building and training neural networks.


---


##from torch.autograd import Variable:
This line imports the Variable class from the torch.autograd module.

In older versions of PyTorch, Variable was used to wrap tensors for automatic differentiation.

However, in newer versions, tensors are automatically differentiable, so this line might not be necessary depending on the PyTorch version being used.

---
#import cv2:
This line imports the OpenCV (cv2) library, which is a powerful computer vision library used for image and video processing.

It provides functions for tasks like reading and writing images, object detection, and image manipulation.

-------
##from data import BaseTransform, VOC_CLASSES as labelmap:
This line imports two objects from a custom module named data.

BaseTransform is likely a class or function that defines a transformation to be applied to images, such as resizing or normalization.

VOC_CLASSES is a list or dictionary containing the names of the object classes used in the Pascal VOC dataset, a popular dataset for object detection.

It is imported and renamed as labelmap for easier use in the code.

----------
from ssd import build_ssd: This line imports the build_ssd function from a custom module named ssd. This function is likely responsible for creating the SSD (Single Shot MultiBox Detector) neural network architecture used for object detection.

import imageio: This line imports the imageio library, which is used for reading and writing various image and video formats. It is used in the code to read the input video and write the output video with the detected objects.

In summary, this snippet imports the necessary libraries and modules for object detection using the SSD architecture. It includes PyTorch for deep learning, OpenCV for computer vision, custom modules for data transformation and network building, and imageio for handling video files.

In [ ]:
# Defining a function that will do the detections
def detect(frame, net, transform):
    height, width = frame.shape[:2]
    frame_t = transform(frame)[0]
    x = torch.from_numpy(frame_t).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))
    y = net(x)
    detections = y.data
    scale = torch.Tensor([width, height, width, height])
    # detections = [batch, number of classes, number of occurence, (score, x0, Y0, x1, y1)]
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.6:
            pt = (detections[0, i, j, 1:] * scale).numpy()
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            j += 1
    return frame

In [ ]:
# Creating the SSD neural network
net = build_ssd('test')
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage))

In [ ]:
# Creating the transformation
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))

In [ ]:
# Doing some Object Detection on a video
reader = imageio.get_reader('funny_dog.mp4')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('output.mp4', fps = fps)
for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame)
    print(i)
writer.close()